In [1]:
%env HSA_OVERRIDE_GFX_VERSION=10.3.0
%env ROCM_PATH=/opt/rocm


env: HSA_OVERRIDE_GFX_VERSION=10.3.0
env: ROCM_PATH=/opt/rocm


In [2]:
from textwrap import wrap
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [3]:
#TODO document jupyter
import pickle
import configparser
import sqlite3 as sl
import pandas as pd
import numpy as np

configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)
datasetPathDatabase =  configParser.get('COMMON', 'datasetPathDatabase') + '/dataset.db'

con = sl.connect(datasetPathDatabase)
data = con.execute("SELECT V.ID, V.VIDEO_PATH, V.AGE, " + 
                    "V.ETHNICITY CAPTION_E, " +
				    "lower(V.GENDER) CAPTION_G, " +
					"A.SPEAKER_EMB, "+ "A.AUDIO_EMB, " +
					"A.LANG CAPTION_L, "+
					"F.FACE_PATH "+
                    "FROM VIDEO V "+
                    "INNER JOIN AUDIO A ON V.ID = A.VIDEO_ID INNER JOIN FACE F ON V.ID = F.VIDEO_ID WHERE AUDIO_PRE = 2 AND FACES_PRE = 1 LIMIT 50000 OFFSET 0")
dataGotten = data.fetchall()

pd.set_option('display.max_columns', None)
df = pd.DataFrame(dataGotten,columns = ['ID','VIDEO_PATH','AGE','caption_e','caption_g','SPEAKER_EMB','AUDIO_EMB','caption_l','image_path'])

def speaker_emb_preprocess(speaker_emb2):

    speaker_emb2 = pickle.loads(speaker_emb2)
    speaker_emb2 = speaker_emb2.squeeze()
    speaker_emb2 = np.pad(speaker_emb2, (288), 'constant', constant_values=(0))
    speaker_emb2 = np.tile(speaker_emb2, (1, 1))
    speaker_emb2 = speaker_emb2 / 200.0
    speaker_emb2 = np.array(speaker_emb2).tolist()
    return speaker_emb2


def audio_emb_preprocess2(speaker_emb2):
    speaker_emb2 = pickle.loads(speaker_emb2)
    a = speaker_emb2 #np.zeros(shape=(24, 512))
    b = np.zeros(shape=(a.shape[0], 768-a.shape[1]))
    c = np.zeros(shape=(50-a.shape[0], 768))
    arr = np.concatenate((a, b), axis=1)
    arr = np.concatenate((arr, c), axis=0)
    arr = arr / 10.0
    speaker_emb2 = np.array(arr).tolist()
    return speaker_emb2

from PIL import Image
def getImage(face_path):
    im = Image.open(face_path)
    im.load() # required for png.split()

    im2 = Image.new("RGB", im.size, (255, 255, 255))
    im2.paste(im, mask=im.split()[3]) # 3 is the alpha channel
    im3 = np.array(im2)
    im4 = np.rollaxis(im3,2)
    return im4



In [4]:
df3 = df[["image_path","caption_e","caption_g","caption_l"]]

In [5]:
import random
def process_gender(gender):
    if(random.random() < 0.2):
        return np.zeros(768)
    elif(gender == 'man'):
        return np.ones(768)
    elif(gender == "woman"):
        return np.ones(768) * -1
    else:
        return np.zeros(768)

In [6]:
# TODO
def process_ethnicity(eth):
    return np.zeros(768)

In [7]:
def process_language(lan):
    x = np.zeros(768)
    if(random.random() < 0.2):
        x = x
    elif(lan == "Arabic"):
        x[0] = 1
    elif(lan == "Portuguese"):
        x[16]=1
    elif(lan == "Romansh_Sursilvan"):
        x[2]=1
    elif(lan == "Japanese"):
        x[3]=1
    elif(lan == "Ukranian"):
        x[4]=1
    elif(lan == "German"):
        x[5]=1   
    elif(lan == "Chinese_China"):
        x[6]=1   
    elif(lan == "Welsh"):
        x[7]=1  
    elif(lan == "English"):
        x[8]=1
    elif(lan == "Kabyle"):
        x[9]=1 
    elif(lan == "Kyrgyz"):
        x[10]=1
    elif(lan == "Georgian"):
        x[11]=1
    elif(lan == "Persian"):
        x[12]=1 
    elif(lan == "French"):
        x[13]=1
    elif(lan == "Interlingua"):
        x[14]=1
    elif(lan == "Swedish"):
        x[15]=1
    elif(lan == "Spanish"):
        x[16]=1 
    elif(lan == "Dhivehi"):
        x[17]=1
    elif(lan == "Kinyarwanda"):
        x[18]=1 
    elif(lan == "Tatar"):
        x[19]=1
    elif(lan == "Hakha_Chin"):
        x[20]=1 
    elif(lan == "Tamil"):
        x[21]=1 
    elif(lan == "Greek"):
        x[22]=1
    elif(lan == "Latvian"):
        x[23]=1 
    elif(lan == "Russian"):
        x[24]=1
    elif(lan == "Breton"):
        x[25]=1
    elif(lan == "Catalan"):
        x[26]=1    
    elif(lan == "Maltese"):
        x[27]=1 
    elif(lan == "Slovenian"):
        x[28]=1    
    elif(lan == "Indonesian"):
        x[29]=1    
    elif(lan == "Dutch"):
        x[30]=1
    elif(lan == "Chinese_Taiwan"):
        x[31]=1 
    elif(lan == "Sakha"):
        x[32]=1 
    elif(lan == "Polish"):
        x[33]=1 
    elif(lan == "Czech"):
        x[34]=1 
    elif(lan == "Romanian"):
        x[35]=1 
    elif(lan == "Mangolian"):
        x[36]=1 
    elif(lan == "Italian"):
        x[37]=1 
    elif(lan == "Chinese_Hongkong"):
        x[38]=1 
    elif(lan == "Estonian"):
        x[39]=1 
    elif(lan == "Basque"):
        x[40]=1 
    elif(lan == "Esperanto"):
        x[41]=1 
    elif(lan == "Frisian"):
        x[42]=1 
    elif(lan == "Turkish"):
        x[43]=1 
    elif(lan == "Chuvash"):
        x[44]=1 
    else:
        x = x
    return x      

In [8]:
process_language("Arabic")

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [9]:
data_frame = df3
data_frame.head()

,image_path,caption_e,caption_g,caption_l
0,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English
1,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English
2,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English
3,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English
4,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English


In [10]:
data_frame['SPEAKER_EMB'] = df['SPEAKER_EMB']
data_frame

/tmp/ipykernel_9663/433991658.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame['SPEAKER_EMB'] = df['SPEAKER_EMB']


,image_path,caption_e,caption_g,caption_l,SPEAKER_EMB
0,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
1,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
2,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
3,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
4,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
...,...,...,...,...,...
364,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
365,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
366,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
367,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...


In [11]:
data_frame['AUDIO_EMB'] = df['AUDIO_EMB']
data_frame

/tmp/ipykernel_9663/1158286622.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame['AUDIO_EMB'] = df['AUDIO_EMB']


,image_path,caption_e,caption_g,caption_l,SPEAKER_EMB,AUDIO_EMB
0,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x8eh\x00\x00\x00\x00\x00\x00\x8...
1,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x8e\xc8\x00\x00\x00\x00\x00\x00...
2,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
3,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x8eh\x00\x00\x00\x00\x00\x00\x8...
4,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x8e\xc8\x00\x00\x00\x00\x00\x00...
...,...,...,...,...,...,...
364,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x8e\xc8\x00\x00\x00\x00\x00\x00...
365,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
366,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x8eh\x00\x00\x00\x00\x00\x00\x8...
367,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x8e\xc8\x00\x00\x00\x00\x00\x00...


In [12]:
pd.options.mode.chained_assignment = None

for index, row in data_frame.iterrows():
    x = speaker_emb_preprocess(data_frame.loc[index,"SPEAKER_EMB"])
    x = [x]
    data_frame.loc[index,"SPEAKER_EMB"] = x




In [13]:
data_frame

,image_path,caption_e,caption_g,caption_l,SPEAKER_EMB,AUDIO_EMB
0,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",b'\x80\x04\x95\x8eh\x00\x00\x00\x00\x00\x00\x8...
1,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",b'\x80\x04\x95\x8e\xc8\x00\x00\x00\x00\x00\x00...
2,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
3,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",b'\x80\x04\x95\x8eh\x00\x00\x00\x00\x00\x00\x8...
4,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",b'\x80\x04\x95\x8e\xc8\x00\x00\x00\x00\x00\x00...
...,...,...,...,...,...,...
364,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",b'\x80\x04\x95\x8e\xc8\x00\x00\x00\x00\x00\x00...
365,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",b'\x80\x04\x95\x84\x00\x00\x00\x00\x00\x00\x00...
366,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",b'\x80\x04\x95\x8eh\x00\x00\x00\x00\x00\x00\x8...
367,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",b'\x80\x04\x95\x8e\xc8\x00\x00\x00\x00\x00\x00...


In [14]:
#CHECK
for index, row in data_frame.iterrows():
    x = audio_emb_preprocess2(data_frame.loc[index,"AUDIO_EMB"])
    x = [x]
    #AADFS = AADFS
    data_frame.loc[index,"AUDIO_EMB"] = x

In [15]:
data_frame

,image_path,caption_e,caption_g,caption_l,SPEAKER_EMB,AUDIO_EMB
0,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.2223130464553833, 0.15062838792800903, 0...."
1,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.2223130464553833, 0.15062838792800903, 0...."
2,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.2223130464553833, 0.15062838792800903, 0...."
3,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.2223130464553833, 0.15062838792800903, 0...."
4,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.2223130464553833, 0.15062838792800903, 0...."
...,...,...,...,...,...,...
364,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.23955292701721193, 0.16991032361984254, 0..."
365,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.23955292701721193, 0.16991032361984254, 0..."
366,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.23955292701721193, 0.16991032361984254, 0..."
367,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,white,man,English,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.23955292701721193, 0.16991032361984254, 0..."


In [16]:
for index, row in data_frame.iterrows():
    x = process_gender(data_frame.loc[index,"caption_g"])
    x = [x]
    #AADFS = AADFS
    data_frame.loc[index,"caption_g"] = x

In [17]:
for index, row in data_frame.iterrows():
    x = process_language(data_frame.loc[index,"caption_l"])
    x = [x]
    #AADFS = AADFS
    data_frame.loc[index,"caption_l"] = x

In [18]:
for index, row in data_frame.iterrows():
    x = process_ethnicity(data_frame.loc[index,"caption_e"])
    x = [x]
    #AADFS = AADFS
    data_frame.loc[index,"caption_e"] = x

In [19]:
data_frame.head(4)

,image_path,caption_e,caption_g,caption_l,SPEAKER_EMB,AUDIO_EMB
0,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.2223130464553833, 0.15062838792800903, 0...."
1,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.2223130464553833, 0.15062838792800903, 0...."
2,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.2223130464553833, 0.15062838792800903, 0...."
3,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.2223130464553833, 0.15062838792800903, 0...."


In [20]:
a = np.zeros(10)
b= np.ones(10)
c = np.vstack((a,b))
np.vstack([c,c])

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [21]:
def process_input(ethnicity,gender,language,speaker_emb,audio_emb):
    speaker_emb = np.asarray(speaker_emb, dtype=np.float32)
    speaker_emb = speaker_emb.squeeze()
    audio_emb = np.asarray(audio_emb, dtype=np.float32)
    audio_emb = audio_emb.squeeze()
    #print(speaker_emb.shape)
    h = np.vstack((ethnicity, gender))
    h = np.vstack((h, language))
    h = np.vstack((h, speaker_emb))
    h = np.vstack((h, audio_emb))

    j = np.zeros(768)
    j = np.tile(j,(256-h.shape[0],1))
    h = np.vstack((h, j))
    #print(h.shape)
    return h.tolist()

In [22]:
data_frame['INPUT'] = data_frame['SPEAKER_EMB']

In [23]:
data_frame

,image_path,caption_e,caption_g,caption_l,SPEAKER_EMB,AUDIO_EMB,INPUT
0,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.2223130464553833, 0.15062838792800903, 0....","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.2223130464553833, 0.15062838792800903, 0....","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.2223130464553833, 0.15062838792800903, 0....","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
3,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.2223130464553833, 0.15062838792800903, 0....","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.2223130464553833, 0.15062838792800903, 0....","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
...,...,...,...,...,...,...,...
364,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.23955292701721193, 0.16991032361984254, 0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
365,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.23955292701721193, 0.16991032361984254, 0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
366,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.23955292701721193, 0.16991032361984254, 0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
367,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[[[0.23955292701721193, 0.16991032361984254, 0...","[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


In [24]:
for index, row in data_frame.iterrows():
    x = process_input(data_frame.loc[index,"caption_e"],data_frame.loc[index,"caption_g"],
                      data_frame.loc[index,"caption_l"],data_frame.loc[index,"SPEAKER_EMB"],
                      data_frame.loc[index,"AUDIO_EMB"])
    x = [x]
    #AADFS = AADFS
    data_frame.loc[index,"INPUT"] = x

In [25]:
data_frame = data_frame.drop(['caption_e', 'caption_g','caption_l','SPEAKER_EMB','AUDIO_EMB'], axis=1)
data_frame

,image_path,INPUT
0,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
3,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,/home/gamal/Datasets/Dataset1/Faces/NZOCucECvD...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
...,...,...
364,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
365,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
366,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
367,/home/gamal/Datasets/Dataset1/Faces/Ol2DffEzNg...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


In [26]:
from PIL import Image
def process_image_path(path):
    
    image = Image.open(path).convert('RGB')
    
    pix = np.array(image,np.float32)
    pix = np.moveaxis(pix, -1, 0)
    
    pix = pix / 255
    return pix.tolist()

In [27]:
for index, row in data_frame.iterrows():
    x = process_image_path(data_frame.loc[index,"image_path"])
    x = [x]
    #AADFS = AADFS
    data_frame.loc[index,"image_path"] = x

In [28]:
np.array(data_frame.head(1)['image_path'][0]).shape

(1, 3, 32, 32)

In [29]:
input = data_frame['INPUT'].to_numpy()
input = np.array([np.array(xi) for xi in input])
input.squeeze().shape

(369, 256, 768)

In [30]:
input = input[:2, :]

In [31]:
import torch
input = torch.from_numpy(input)
input = input.to(torch.float)

In [32]:
output = data_frame['image_path'].to_numpy()
output = np.array([np.array(xi) for xi in output])
output.squeeze().shape

(369, 3, 32, 32)

In [33]:
output = output[:2, :]

In [34]:
output = torch.from_numpy(output)
output = output.to(torch.float)

In [35]:
input

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])

In [36]:
output

tensor([[[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]]],



        [[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,

In [37]:
input = input.squeeze()

In [38]:
output = output.squeeze()

In [39]:
from torch.utils.data import TensorDataset, DataLoader
my_dataset = TensorDataset(output,input) # create your datset
my_dataloader = DataLoader(my_dataset) # create your dataloader

import torch
from imagen_pytorch import Unet, Imagen, ImagenTrainer

# unet for imagen



unet1 = Unet(
    dim = 32,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = 3,
    layer_attns = (False, True, True, True),
)


# imagen, which contains the unets above (base unet and super resoluting ones)

imagen = Imagen(
    unets = (unet1),
    image_sizes = (32),
    timesteps = 10000000,
    cond_drop_prob = 0.1
).cuda()

# wrap imagen with the trainer class

trainer = ImagenTrainer(imagen)

# mock images (get a lot of this) and text encodings from large T5

text_embeds = input.cuda()
images = output.cuda()

# feed images into imagen, training each unet in the cascade

loss = trainer(
    images,
    text_embeds = text_embeds,
    unet_number = 1,            # training on unet number 1 in this example, but you will have to also save checkpoints and then reload and continue training on unet number 2
    max_batch_size = 1          # auto divide the batch of 64 up into batch size of 4 and accumulate gradients, so it all fits in memory
)

trainer.update(unet_number = 1)

trainer.los

# do the above for many many many many steps
# now you can sample an image based on the text embeddings from the cascading ddpm

images = trainer.sample(text_embeds = text_embeds ,cond_scale = 3.)

images.shape # (2, 3, 256, 256)

In [41]:
from imagen_pytorch import Unet, Imagen, ImagenTrainer
from imagen_pytorch.data import Dataset
from imagen_pytorch.t5 import t5_encode_text
t5_encode_text(texts="Hi how have you been my love, what is the largest embedding that I can get here in egypt, is there any bigger or smaller than this i wonder").max()

tensor(1.9394, device='cuda:0')

In [42]:
input.max()

tensor(1.)

In [43]:
from imagen_pytorch import Unet, Imagen, ImagenTrainer
from imagen_pytorch.data import Dataset

# unets for unconditional imagen

unet = Unet(
    dim = 32,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = 1,
    layer_attns = (False, False, False, True),
    layer_cross_attns = False
)

# imagen, which contains the unet above

imagen = Imagen(
    unets = unet,
    image_sizes = 32,
    timesteps = 1000
)

trainer = ImagenTrainer(
    imagen = imagen,
    split_valid_from_train = True # whether to split the validation dataset from the training
).cuda()

# instantiate your dataloader, which returns the necessary inputs to the DDPM as tuple in the order of images, text embeddings, then text masks. in this case, only images is returned as it is unconditional training



trainer.add_train_dataset(my_dataset, batch_size = 1)

# working training loop

for i in range(200000):
    loss = trainer.train_step(unet_number = 1, max_batch_size = 1)
    print(f'loss: {loss}')

    if not (i % 50):
        valid_loss = trainer.valid_step(unet_number = 1, max_batch_size = 1)
        print(f'valid loss: {valid_loss}')

    if not (i % 100) and trainer.is_main: # is_main makes sure this can run in distributed
        images = trainer.sample(text_embeds=input,batch_size = 1, return_pil_images = True) # returns List[Image]
        images[0].save(f'imagen-samples/sample-{i // 100}.png')

The base dimension of your u-net should ideally be no smaller than 128, as recommended by a professional DDPM trainer https://nonint.com/2022/05/04/friends-dont-let-friends-train-small-diffusion-models/
training with dataset of 1 samples and validating with randomly splitted 1 samples
loss: 0.467155784368515
valid loss: 0.8219684958457947


0it [00:00, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

loss: 0.6266807317733765
loss: 0.15448665618896484
loss: 0.20638182759284973
loss: 0.9565389752388
loss: 0.6960496306419373
loss: 0.13171511888504028
loss: 0.171579971909523
loss: 0.21935702860355377
loss: 0.8149603009223938
loss: 0.9700176119804382
loss: 0.8566367030143738
loss: 0.9709348082542419
loss: 0.8859018683433533
loss: 0.6416133642196655
loss: 0.5212737917900085
loss: 0.9190928339958191
loss: 0.8517638444900513
loss: 0.3943752646446228
loss: 0.6369068026542664
loss: 0.6439350247383118
loss: 0.6106128096580505
loss: 0.23307384550571442
loss: 0.8307128548622131
loss: 0.49635910987854004
loss: 0.7209218144416809
loss: 0.8210404515266418
loss: 0.6803538203239441
loss: 0.775252640247345
loss: 0.7777044773101807
loss: 0.5798169374465942
loss: 0.2159399688243866
loss: 0.8342316746711731
loss: 0.7082736492156982
loss: 0.6388343572616577
loss: 0.8131501078605652
loss: 0.5006447434425354
loss: 0.013680755160748959
loss: 0.6212562322616577
loss: 0.6412934064865112
loss: 0.85749268531799

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

loss: 0.00805257260799408
loss: 0.0061760954558849335
loss: 0.007524489425122738
loss: 0.006069212220609188
loss: 0.0071237292140722275
loss: 0.006843621376901865
loss: 0.006494318600744009
loss: 0.006547980010509491
loss: 0.009341111406683922
loss: 0.006727979984134436
loss: 0.007220232393592596
loss: 0.015025191940367222
loss: 0.0062570213340222836
loss: 0.006002925802022219
loss: 0.009696047753095627
loss: 0.005976513493806124
loss: 0.005026807077229023
loss: 0.009236998856067657
loss: 0.006934621371328831
loss: 0.007483146619051695
loss: 0.00817228201776743
loss: 0.0056662349961698055
loss: 0.005220260005444288
loss: 0.01027235109359026
loss: 0.007582954131066799
loss: 0.008335404098033905
loss: 0.008655858226120472
loss: 0.007709708996117115
loss: 0.005715483799576759
loss: 0.005502510815858841
loss: 0.009663308970630169
loss: 0.006577176507562399
loss: 0.0071857343427836895
loss: 0.005849048029631376
loss: 0.012180078774690628
loss: 0.005359157454222441
loss: 0.006202194839715958

0it [00:00, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
trainer.save('imagen.pt')

checkpoint saved to imagen.pt


In [ ]:
image = images[1]

In [ ]:
image = image.cpu().detach().numpy()

In [ ]:
image = np.moveaxis(image, 0, -1)

In [ ]:
image = image * 255

In [ ]:
image = Image.fromarray(image.astype('uint8'), 'RGB')

In [ ]:
image.save('a.png')